<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold">
<br>NEXT TONNE - BKGND INDEX</p><br>

In [2]:
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
# General importings
import math
import numpy             as np
import tables            as tb
import pandas            as pd
import matplotlib.pyplot as plt

# IC stuff
import invisible_cities.core.system_of_units as units

# TONNE stuff
from detector_dimensions  import get_dimensions
from detector_dimensions  import print_dimensions

from initial_activities  import get_radiogenic_activities
from initial_activities  import get_radon_activity
from initial_activities  import get_muon_flux
from initial_activities  import get_muon_flux_error
from initial_activities  import print_initial_activities

from detector_backgrounds import get_radiogenic_background_level
from detector_backgrounds import get_radon_background_level
from detector_backgrounds import get_muon_background_level

from rejection_factors    import get_rejection_factors
from roi_settings         import get_roi_settings


# DETAILS OF ANALYSIS

In [45]:
# Available detectors: "next_2x2", "next_3x3", "next_hd"
DETECTOR = 'next_hd'

# Available radiogenic background levels: "reference", "probable", "optimistic"
RADIOGENIC_BKGND_LEVEL = 'probable'

# Available Radon background levels: "pessimistic", "optimistic"
RADON_BKGND_LEVEL = 'pessimistic'

# Hosting lab: 'LSC', 'LNGS', 'SNOLAB'
HOSTING_LAB = 'SNOLAB'

# LOADING DETECTOR DATA

In [46]:
det_dim = get_dimensions(DETECTOR)

In [47]:
print_dimensions(DETECTOR)

*** DETECTOR 'next_hd' DIMENSIONS ***

* ACTIVE
  ACTIVE Diameter = 260.0 cm
  ACTIVE Length   = 260.0 cm
  ACTIVE Surface  = 3.186e+05 cm**2
  ACTIVE Volume   = 1.380e+07 cm**3
  ACTIVE mass     = 1.229e+03 kg

* READOUT_PLANE
  READOUT_PLANE Surface (each one) = 5.309e+04 cm**2

* CATHODE
  CATHODE Volume = 1.327e+03 cm**3
  CATHODE Mass   = 1.061e+01 kg

* FIELD_CAGE
  FIELD_CAGE Thickness = 1.0 cm
  FIELD_CAGE Inner Rad = 1.300e+02 cm
  FIELD_CAGE Outer Rad = 1.310e+02 cm
  FIELD_CAGE Length    = 2.600e+02 cm
  FIELD_CAGE Volume    = 2.132e+05 cm**3
  FIELD_CAGE Mass      = 4.690e+02 kg

* INNER_SHIELDING
  ICS Thickness         = 12.0 cm
  ICS Inner Rad         = 1.310e+02 cm
  ICS Outer Rad         = 1.430e+02 cm
  ICS Inner Length      = 2.640e+02 cm
  ICS Outer Length      = 2.880e+02 cm
  ICS Inner EndCap Surf = 5.391e+04 cm**2
  ICS Outer EndCap Surf = 6.424e+04 cm**2
  ICS Volume            = 4.269e+06 cm**3
  ICS Mass              = 3.825e+04 kg

* VESSEL
  VESSEL Thickness

In [48]:
radiogenic_act = get_radiogenic_activities(RADIOGENIC_BKGND_LEVEL)
radon_act      = get_radon_activity(RADON_BKGND_LEVEL)
muon_flux      = get_muon_flux(HOSTING_LAB)

In [49]:
print_initial_activities(RADIOGENIC_BKGND_LEVEL, RADON_BKGND_LEVEL, HOSTING_LAB)

*** 'probable' RADIOGENIC ACTIVITIES ***

* Copper:
    Bi214: 1.260e-03 mBq/kg
    Tl208: 4.350e-04 mBq/kg

* DiceBoard:
    Bi214: 4.620e-02 mBq/m**2
    Tl208: 1.223e-02 mBq/m**2

* Teflon:
    Bi214: 4.960e-03 mBq/kg
    Tl208: 3.690e-05 mBq/kg


*** 'pessimistic' RADON ACTIVITY ***

* Radon activity: 2.910e+00 mBq/m**2


*** 'SNOLAB' MUON FLUX ***

* Muon flux: 3.310e-10 cm**-2 sec**-1


# COMPUTING THE DETECTOR BACKGROUND LEVEL

#### Radiogenic contributions

In principle. only the contributions from the 'READOUT_PLANES', 'FIELD_CAGE' and 'INNER_SHIELDING' are considered.

Backgrounds expressed in Becquerel

In [50]:
radiogenic_bkgnd = get_radiogenic_background_level(DETECTOR, RADIOGENIC_BKGND_LEVEL)
radiogenic_bkgnd['Total'] = radiogenic_bkgnd['Bi214'] + radiogenic_bkgnd['Tl208']
print(f"*** '{DETECTOR}' radiogenic background level (Bq.):")
radiogenic_bkgnd

*** 'next_hd' radiogenic background level (Bq.):


,Tl208,Bi214,Total
source,,,
READOUT_PLANE,0.000130,0.000491,0.000620
FIELD_CAGE,0.000017,0.002326,0.002344
INNER_SHIELDING,0.016638,0.048193,0.064831


In [51]:
tot_bkgnd = radiogenic_bkgnd['Total'].sum()
print(f"*** '{DETECTOR}' radiogenic background: {tot_bkgnd:8.3e} Bq.")

tot_Bi = radiogenic_bkgnd['Bi214'].sum()
tot_Tl = radiogenic_bkgnd['Tl208'].sum()
print(f"\nTotal Bi214 background: {tot_Bi:8.3e} Bq.  ->  {tot_Bi/tot_bkgnd*100:.4} %")
print(f"Total Tl208 background: {tot_Tl:8.3e} Bq.  ->  {tot_Tl/tot_bkgnd*100:.4} %")

tot_READOUT_PLANE = radiogenic_bkgnd.loc['READOUT_PLANE'].Total
tot_FIELD_CAGE    = radiogenic_bkgnd.loc['FIELD_CAGE'].Total
tot_ICS           = radiogenic_bkgnd.loc['INNER_SHIELDING', 'Total']
print(f"\nTotal READOUT_PLANE background: {tot_READOUT_PLANE:8.3e} Bq.  ->  {tot_READOUT_PLANE/tot_bkgnd*100:4.2} %")
print(f"Total FIELD_CAGE background:    {tot_FIELD_CAGE:8.3e} Bq.  ->  {tot_FIELD_CAGE/tot_bkgnd*100:.4} %")
print(f"Total ICS background:           {tot_ICS:8.3e} Bq.  ->  {tot_ICS/tot_bkgnd*100:.4} %")


*** 'next_hd' radiogenic background: 6.780e-02 Bq.

Total Bi214 background: 5.101e-02 Bq.  ->  75.24 %
Total Tl208 background: 1.679e-02 Bq.  ->  24.76 %

Total READOUT_PLANE background: 6.204e-04 Bq.  ->  0.92 %
Total FIELD_CAGE background:    2.344e-03 Bq.  ->  3.457 %
Total ICS background:           6.483e-02 Bq.  ->  95.63 %


#### Radon contribution

The radon contribution is directly translated to an Bi214 concentration in the central CATHODE

In [52]:
radon_background = get_radon_background_level(DETECTOR, RADON_BKGND_LEVEL)
print(f"*** '{DETECTOR}' radon background: {radon_background/units.Bq:8.3e} Bq.")
print(f"\n(It will be treated as Bi214 contamination from the CATHODE)")

*** 'next_hd' radon background: 9.270e-02 Bq.

(It will be treated as Bi214 contamination from the CATHODE)


#### Muons contribution

Xe137 decay is the only muon contribution considered.

In [53]:
muon_background, muon_background_error = get_muon_background_level(DETECTOR, HOSTING_LAB)
print(f"\n\n*** '{DETECTOR}' in '{HOSTING_LAB}' muon background: {muon_background:8.3e} Bq.")
print(f"\n(It will be treated as Xe137 contamination from the ACTIVE)")

Xe-137 per second =  3.81249273045336e-08 +/- 7.466892601995566e-10
Xe-137 per calendar yr =  1.2023077074757718 +/- 0.023547592509653215


*** 'next_hd' in 'SNOLAB' muon background: 3.812e-08 Bq.

(It will be treated as Xe137 contamination from the ACTIVE)


# LOADING & SELECTING REJECTION FACTORS

In [54]:
rej_factors = get_rejection_factors(DETECTOR)
print(f"*** '{DETECTOR}' rejection factors:")
rej_factors

*** 'next_hd' rejection factors:


bb0nu  bb0nu_err     Xe137  \
source          energyRes spatialDef                                  
ACTIVE          0.7       3x3x3       0.237636   0.000450  0.000075   
                          10x10x10    0.285822   0.000487  0.000147   
                0.5       3x3x3       0.245923   0.000454  0.000057   
                          10x10x10    0.295388   0.000496  0.000113   
READOUT_PLANE   0.7       3x3x3            NaN        NaN       NaN   
                          10x10x10         NaN        NaN       NaN   
                0.5       3x3x3            NaN        NaN       NaN   
                          10x10x10         NaN        NaN       NaN   
CATHODE         0.7       3x3x3            NaN        NaN       NaN   
                          10x10x10         NaN        NaN       NaN   
                0.5       3x3x3            NaN        NaN       NaN   
                          10x10x10         NaN        NaN       NaN   
FIELD_CAGE      0.7       3x3x3            NaN        NaN       NaN   
                          10x10x10         NaN        NaN       NaN   
                0.5       3x3x3            NaN        NaN       NaN   
                          10x10x10         NaN        NaN       NaN   
INNER_SHIELDING 0.7       3x3x3            NaN        NaN       NaN   
                          10x10x10         NaN        NaN       NaN   
                0.5       3x3x3            NaN        NaN       NaN   
                          10x10x10         NaN        NaN       NaN   

                                      Xe137_err         Bi214     Bi214_err  \
source          energyRes spatialDef                                          
ACTIVE          0.7       3x3x3        0.000002           NaN           NaN   
                          10x10x10     0.000003           NaN           NaN   
                0.5       3x3x3        0.000002           NaN           NaN   
                          10x10x10     0.000003           NaN           NaN   
READOUT_PLANE   0.7       3x3x3             NaN  1.561224e-07  1.262175e-08   
                          10x10x10          NaN  4.650000e-07  2.156386e-08   
                0.5       3x3x3             NaN  7.807808e-08  8.840601e-09   
                          10x10x10          NaN  2.629969e-07  1.637347e-08   
CATHODE         0.7       3x3x3             NaN  0.000000e+00  0.000000e+00   
                          10x10x10          NaN  3.005410e-09  1.735174e-09   
                0.5       3x3x3             NaN  1.002406e-09  1.002406e-09   
                          10x10x10          NaN  1.003814e-09  1.003814e-09   
FIELD_CAGE      0.7       3x3x3             NaN  1.416838e-07  1.206092e-08   
                          10x10x10          NaN  5.010000e-07  2.238303e-08   
                0.5       3x3x3             NaN  6.300000e-08  7.937254e-09   
                          10x10x10          NaN  2.320329e-07  1.543460e-08   
INNER_SHIELDING 0.7       3x3x3             NaN  1.951366e-08  1.397402e-09   
                          10x10x10          NaN  6.852741e-08  2.618298e-09   
                0.5       3x3x3             NaN  1.160348e-08  1.077356e-09   
                          10x10x10          NaN  3.292634e-08  1.815288e-09   

                                             Tl208     Tl208_err  
source          energyRes spatialDef                              
ACTIVE          0.7       3x3x3                NaN           NaN  
                          10x10x10             NaN           NaN  
                0.5       3x3x3                NaN           NaN  
                          10x10x10             NaN           NaN  
READOUT_PLANE   0.7       3x3x3       9.588048e-08  1.033906e-08  
                          10x10x10    3.288889e-07  1.911628e-08  
                0.5       3x3x3       6.779284e-08  8.679984e-09  
                          10x10x10    2.442152e-07  1.650254e-08  
CATHODE         0.7       3x3x3                NaN           NaN  
  

### OPTIONS TO STUDY

In [55]:
energyRes  = 0.5
spatialDef = '3x3x3'

#### Signal efficiency

In [56]:
sig_eff     = rej_factors.loc[pd.IndexSlice['ACTIVE', energyRes, spatialDef], 'bb0nu']
sig_eff_err = rej_factors.loc[pd.IndexSlice['ACTIVE', energyRes, spatialDef], 'bb0nu_err']
print(f"*** '{DETECTOR}' - Energy Res: {energyRes}% - Spatial Def: {spatialDef}\n")
print(f"Signal efficiency:  {sig_eff:.3} +- {sig_eff_err:.3} ")

*** 'next_hd' - Energy Res: 0.5% - Spatial Def: 3x3x3

Signal efficiency:  0.246 +- 0.000454 


#### Radiogenic rejection_factors

In [57]:
radiogenic_rejection = rej_factors.loc[pd.IndexSlice[:, energyRes, spatialDef], ['Bi214', 'Tl208']]
radiogenic_rejection.drop('ACTIVE',  level=0, axis=0, inplace=True)
radiogenic_rejection.drop('CATHODE', level=0, axis=0, inplace=True)
print(f"*** '{DETECTOR}' - Energy Res: {energyRes}% - Spatial Def: {spatialDef}\n")
print(f"Radiogenic Rejection factor:")
radiogenic_rejection

*** 'next_hd' - Energy Res: 0.5% - Spatial Def: 3x3x3

Radiogenic Rejection factor:


,,,Bi214,Tl208
source,energyRes,spatialDef,,
READOUT_PLANE,0.5,3x3x3,7.807808e-08,6.779284e-08
FIELD_CAGE,0.5,3x3x3,6.300000e-08,6.777778e-08
INNER_SHIELDING,0.5,3x3x3,1.160348e-08,4.500000e-08


In [58]:
radiogenic_rejection_error = rej_factors.loc[pd.IndexSlice[:, energyRes, spatialDef], ['Bi214_err', 'Tl208_err']]
radiogenic_rejection_error.drop('ACTIVE',  level=0, axis=0, inplace=True)
radiogenic_rejection_error.drop('CATHODE', level=0, axis=0, inplace=True)
print(f"*** '{DETECTOR}' - Energy Res: {energyRes}% - Spatial Def: {spatialDef}\n")
print(f"Radiogenic Rejection factor error:")
radiogenic_rejection_error

*** 'next_hd' - Energy Res: 0.5% - Spatial Def: 3x3x3

Radiogenic Rejection factor error:


,,,Bi214_err,Tl208_err
source,energyRes,spatialDef,,
READOUT_PLANE,0.5,3x3x3,8.840601e-09,8.679984e-09
FIELD_CAGE,0.5,3x3x3,7.937254e-09,8.678055e-09
INNER_SHIELDING,0.5,3x3x3,1.077356e-09,6.708204e-09


#### Radon rejection_factor

As in previos NEXT detectors we assume that the whole amount of Radon contamination is translated to Bi214 contamination from the CATHODE.

In [59]:
radon_rejection     = rej_factors.loc[pd.IndexSlice['CATHODE', energyRes, spatialDef], 'Bi214']
radon_rejection_err = rej_factors.loc[pd.IndexSlice['CATHODE', energyRes, spatialDef], 'Bi214_err']
print(f"*** '{DETECTOR}' - Energy Res: {energyRes}% - Spatial Def: {spatialDef}\n")
print(f"Radon rejection factor:  {radon_rejection:8.3e} +- {radon_rejection_err:8.3e}")

*** 'next_hd' - Energy Res: 0.5% - Spatial Def: 3x3x3

Radon rejection factor:  1.002e-09 +- 1.002e-09


#### Muons rejection_factors

The muons rejection factor = Xe137_activation * Xe137_rejection.

As the Xe137_activation has been considered previously, the Xe137_rejection factor is the only one applied here.

In [60]:
Xe137_rejection     = rej_factors.loc[pd.IndexSlice['ACTIVE', energyRes, spatialDef], 'Xe137']
Xe137_rejection_err = rej_factors.loc[pd.IndexSlice['ACTIVE', energyRes, spatialDef], 'Xe137_err']
print(f"*** '{DETECTOR}' - Energy Res: {energyRes}% - Spatial Def: {spatialDef}\n")
print(f"Xe137 rejection factor:  {Xe137_rejection:8.3e} +- {Xe137_rejection_err:8.3e}")

*** 'next_hd' - Energy Res: 0.5% - Spatial Def: 3x3x3

Xe137 rejection factor:  5.681e-05 +- 1.732e-06


# COMPUTING THE DETECTOR BACKGROUND INDEX

#### Estimating the Bq -> ckky conversion factor 

In [61]:
secs_in_year    = 60 * 60 * 24 * 365
Xe136_abundance = 0.902616
Xe136_mass_kg   = det_dim['ACTIVE_mass'] * Xe136_abundance / units.kg
ROI_settings    = get_roi_settings(energyRes)

In [62]:
toCKKY = secs_in_year / Xe136_mass_kg / ((ROI_settings['Emax'] - ROI_settings['Emin']) / units.keV)

In [63]:
print(f"*** '{DETECTOR}' - Energy Res: {energyRes}% - Spatial Def: {spatialDef}\n")
print(f"Xe136 mass: {Xe136_mass_kg:.6} kg.")
print(f"ROI: [{ROI_settings['Emin']/units.keV}, {ROI_settings['Emax']/units.keV}] keV.")
print(f"toCKKY factor: {toCKKY:.6}")

*** 'next_hd' - Energy Res: 0.5% - Spatial Def: 3x3x3

Xe136 mass: 1108.93 kg.
ROI: [2454.0, 2471.0] keV.
toCKKY factor: 1672.84


#### Radiogenic background index

In [64]:
radiogenic_bkgnd_index_Bq = radiogenic_bkgnd[['Bi214', 'Tl208']] * radiogenic_rejection[['Bi214', 'Tl208']]
radiogenic_bkgnd_index_Bq['Total'] = radiogenic_bkgnd_index_Bq['Bi214'].add(radiogenic_bkgnd_index_Bq['Tl208'],
                                                                            fill_value = 0.)
radiogenic_bkgnd_index = radiogenic_bkgnd_index_Bq * toCKKY
print(f"*** '{DETECTOR}' - Energy Res: {energyRes}% - Spatial Def: {spatialDef}\n")
print(f"Radiogenic Backgroun Index (ckky):")
radiogenic_bkgnd_index

*** 'next_hd' - Energy Res: 0.5% - Spatial Def: 3x3x3

Radiogenic Backgroun Index (ckky):


,,,Bi214,Tl208,Total
source,energyRes,spatialDef,,,
READOUT_PLANE,0.5,3x3x3,6.407556e-08,1.472759e-08,7.880314e-08
FIELD_CAGE,0.5,3x3x3,2.451678e-07,1.962252e-09,2.471300e-07
INNER_SHIELDING,0.5,3x3x3,9.354693e-07,1.252485e-06,2.187954e-06


In [65]:
radiogenic_bkgnd_index_error_Bq = pd.DataFrame(radiogenic_bkgnd[['Bi214', 'Tl208']].values * \
                                               radiogenic_rejection_error[['Bi214_err', 'Tl208_err']].values,
                                               index = radiogenic_bkgnd.index, columns=['Bi214_err', 'Tl208_err'])
radiogenic_bkgnd_index_error = radiogenic_bkgnd_index_error_Bq * toCKKY
print(f"*** '{DETECTOR}' - Energy Res: {energyRes}% - Spatial Def: {spatialDef}\n")
print(f"Radiogenic Backgroun Index Statistical Error (ckky):")
radiogenic_bkgnd_index_error

*** 'next_hd' - Energy Res: 0.5% - Spatial Def: 3x3x3

Radiogenic Backgroun Index Statistical Error (ckky):


,Bi214_err,Tl208_err
source,,
READOUT_PLANE,7.255128e-09,1.885674e-09
FIELD_CAGE,3.088823e-08,2.512407e-10
INNER_SHIELDING,8.685615e-08,1.867094e-07


In [66]:
print(f"*** '{DETECTOR}' - Energy Res: {energyRes}% - Spatial Def: {spatialDef}\n")
tot_radiogenic_bkgnd_index_Bq = radiogenic_bkgnd_index_Bq['Total'].sum()
print(f"Radiogenic background level: {tot_radiogenic_bkgnd_index_Bq:8.3e} Bq.")
tot_radiogenic_bkgnd_index = radiogenic_bkgnd_index['Total'].sum()
print(f"Radiogenic background index: {tot_radiogenic_bkgnd_index:8.3e} ckky")

tot_Bi = radiogenic_bkgnd_index['Bi214'].sum()
tot_Tl = radiogenic_bkgnd_index['Tl208'].sum()
print(f"\nTotal Bi214         background index: {tot_Bi:8.3e} ckky.  ->  {tot_Bi/tot_radiogenic_bkgnd_index*100:6.4} %")
print(f"Total Tl208         background index: {tot_Tl:8.3e} ckky.  ->  {tot_Tl/tot_radiogenic_bkgnd_index*100:6.4} %")

tot_READOUT_PLANE = radiogenic_bkgnd_index.loc[pd.IndexSlice['READOUT_PLANE', energyRes, spatialDef]].Total
tot_FIELD_CAGE    = radiogenic_bkgnd_index.loc[pd.IndexSlice['FIELD_CAGE', energyRes, spatialDef]].Total
tot_ICS           = radiogenic_bkgnd_index.loc[pd.IndexSlice['INNER_SHIELDING', energyRes, spatialDef]].Total
print(f"\nTotal READOUT_PLANE background index: {tot_READOUT_PLANE:8.3e} ckky.  ->  {tot_READOUT_PLANE/tot_radiogenic_bkgnd_index*100:6.4} %")
print(f"Total FIELD_CAGE    background index: {tot_FIELD_CAGE:8.3e} ckky.  ->  {tot_FIELD_CAGE/tot_radiogenic_bkgnd_index*100:6.4} %")
print(f"Total ICS           background index: {tot_ICS:8.3e} ckky.  ->  {tot_ICS/tot_radiogenic_bkgnd_index*100:6.4} %")


*** 'next_hd' - Energy Res: 0.5% - Spatial Def: 3x3x3

Radiogenic background level: 1.503e-09 Bq.
Radiogenic background index: 2.514e-06 ckky

Total Bi214         background index: 1.245e-06 ckky.  ->   49.51 %
Total Tl208         background index: 1.269e-06 ckky.  ->   50.49 %

Total READOUT_PLANE background index: 7.880e-08 ckky.  ->   3.135 %
Total FIELD_CAGE    background index: 2.471e-07 ckky.  ->   9.831 %
Total ICS           background index: 2.188e-06 ckky.  ->   87.03 %


#### Radon background index

In [67]:
radon_bkgnd_index_Bq = (radon_background / units.Bq) * radon_rejection
radon_bkgnd_index    = radon_bkgnd_index_Bq * toCKKY

radon_bkgnd_index_error_Bq = (radon_background / units.Bq) * radon_rejection_err
radon_bkgnd_index_error    = radon_bkgnd_index_error_Bq * toCKKY

print(f"*** '{DETECTOR}' - Energy Res: {energyRes}% - Spatial Def: {spatialDef}\n")
print(f"Radon background index: {radon_bkgnd_index:8.3e}  (+- {radon_bkgnd_index_error:8.3e}) ckky")

*** 'next_hd' - Energy Res: 0.5% - Spatial Def: 3x3x3

Radon background index: 1.554e-07  (+- 1.554e-07) ckky


#### Muons background index

In [68]:
muon_bkgnd_index_Bq = muon_background * Xe137_rejection
muon_bkgnd_index    = muon_bkgnd_index_Bq * toCKKY

muon_bkgnd_index_error_Bq = muon_background * Xe137_rejection_err
muon_bkgnd_index_error    = muon_bkgnd_index_error_Bq * toCKKY

print(f"*** '{DETECTOR}' - Energy Res: {energyRes}% - Spatial Def: {spatialDef}\n")
print(f"Muon background index: {muon_bkgnd_index:8.3e}  (+- {muon_bkgnd_index_error:8.3e}) ckky")

*** 'next_hd' - Energy Res: 0.5% - Spatial Def: 3x3x3

Muon background index: 3.623e-09  (+- 1.105e-10) ckky


### DETECTOR SUMMARY

In [43]:
bkgnd_index_Bq = tot_radiogenic_bkgnd_index_Bq + radon_bkgnd_index_Bq + muon_bkgnd_index_Bq
print(f"Total background level: {bkgnd_index_Bq:8.3e} Bq.")
bkgnd_index    = tot_radiogenic_bkgnd_index    + radon_bkgnd_index    + muon_bkgnd_index
print(f"Total background index: {bkgnd_index:8.3e} ckky")

Total background level: 1.800e-09 Bq.
Total background index: 3.011e-06 ckky


In [44]:
print(f"***** '{DETECTOR}' *****\n")

print(f"** Radiogenic background level: {RADIOGENIC_BKGND_LEVEL}")
print(f"** Radon background level:      {RADON_BKGND_LEVEL}")
print(f"** Hosting laboratory:          {HOSTING_LAB}\n")

print(f"** Energy Resolution:  {energyRes}")
print(f"** Spatial definition: {spatialDef}\n")


print(f"   Xe136 mass:        {Xe136_mass_kg:8.3e} Kg")
print(f"   Signal efficiency: {sig_eff:.3}\n")

print(f"   Radiogenic bkgnd index: {tot_radiogenic_bkgnd_index:8.3e} ckky")
print(f"   Radon      bkgnd index: {radon_bkgnd_index:8.3e} ckky")
print(f"   Muons      bkgnd index: {muon_bkgnd_index:8.3e} ckky\n")

print(f"   Total      bkgnd index: {bkgnd_index:8.3e} ckky\n")

print(f"   FIGURE OF MERIT (s/sqrt(b)): {sig_eff/math.sqrt(bkgnd_index_Bq):8.3e} a.u.")

***** 'next_hd' *****

** Radiogenic background level: probable
** Radon background level:      pessimistic
** Hosting laboratory:          LNGS

** Energy Resolution:  0.5
** Spatial definition: 3x3x3

   Xe136 mass:        1.109e+03 Kg
   Signal efficiency: 0.246

   Radiogenic bkgnd index: 2.514e-06 ckky
   Radon      bkgnd index: 1.554e-07 ckky
   Muons      bkgnd index: 3.417e-07 ckky

   Total      bkgnd index: 3.011e-06 ckky

   FIGURE OF MERIT (s/sqrt(b)): 5.797e+03 a.u.


# SENSITIVITY

In [ ]:
#from pybbsens import isotope

In [70]:
3.011e-06 * 17 * 1.109e+03

0.056766383000000004

In [71]:
3.011e-06 * 17 * 1.e+03

0.051187